In [ ]:
import os
from glob import glob
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

import pydicom
from pymedphys.deliverydata import DeliveryData
from pymedphys.dcm import anonymise_dicom, coords_and_dose_from_dcm
from pymedphys.mudensity import find_relevant_control_points, calc_mu_density, get_grid
from pymedphys.logfile import delivery_data_from_logfile
from pymedphys.gamma import gamma_dcm, convert_to_percent_pass

from pymedphys.plt import pcolormesh_grid

In [ ]:
data_root = r'C:\Users\sbiggs\logfile2dicom\endometrium'

In [ ]:
logfile_40ms_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*LogfileCalc40msLow*.dcm'
))[0]

logfile_40ms_dose_dicom_path

In [ ]:
# logfile_240ms_dose_dicom_path = glob(os.path.join(
#     data_root, 'calced_dose', '*LogfileCalc240msLow*.dcm'
# ))[0]

# logfile_240ms_dose_dicom_path

In [ ]:
patient_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*OneBeam*.dcm'
))[0]

patient_dose_dicom_path

In [ ]:
# dose_percent_threshold = 2
# distance_threshold = 2

# gamma = gamma_dcm(
#     logfile_240ms_dose_dicom_path, patient_dose_dicom_path,
#     dose_percent_threshold, distance_threshold, local_gamma=True, random_subset=10000, max_gamma=1.1)

# valid_gamma = gamma[~np.isnan(gamma)]
# print(np.mean(valid_gamma < 1))

# num_points = len(valid_gamma)
# weights = np.ones(num_points)/num_points

# plt.hist(valid_gamma, 20, weights=weights)
# plt.title('Gamma histogram {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
# plt.xlabel('Local Gamma')
# plt.ylabel('Normalised bin count')

# max_x_axis = np.ceil(np.max(valid_gamma))

# plt.xlim([0, max_x_axis])
# plt.show()

In [ ]:
# dose_percent_threshold = 2
# distance_threshold = 2


# gamma = gamma_dcm(
#     logfile_40ms_dose_dicom_path, patient_dose_dicom_path, 
#     dose_percent_threshold, distance_threshold, local_gamma=True)

# valid_gamma = gamma[~np.isnan(gamma)]
# print(np.mean(valid_gamma < 1))

# num_points = len(valid_gamma)
# weights = np.ones(num_points)/num_points

# plt.hist(valid_gamma, 20, weights=weights)
# plt.title('Gamma histogram {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
# plt.xlabel('Local Gamma')
# plt.ylabel('Normalised bin count')

# max_x_axis = np.ceil(np.max(valid_gamma))

# plt.xlim([0, max_x_axis])
# plt.show()

In [ ]:
coords_reference, dose_reference = coords_and_dose_from_dcm(
    patient_dose_dicom_path)
coords_evaluation_240, dose_evaluation_240 = coords_and_dose_from_dcm(
    logfile_240ms_dose_dicom_path)

coords_evaluation_40, dose_evaluation_40 = coords_and_dose_from_dcm(
    logfile_40ms_dose_dicom_path)

In [ ]:
keep = dose_reference > 0.2*np.max(dose_reference)
num_points = np.sum(keep)
weights = np.ones(num_points)/num_points

diff_240 = (dose_evaluation_240 - dose_reference)[keep]
diff_40 = (dose_evaluation_40 - dose_reference)[keep]

In [ ]:
# plt.hist(diff_40, 100, weights=weights)
# np.std(diff_40)

In [ ]:
# plt.hist(diff_240, 100, weights=weights)
# np.std(diff_240)

In [ ]:
dose_percent_threshold = 2
distance_threshold = 1


gamma = gamma_dcm(
    patient_dose_dicom_path, logfile_40ms_dose_dicom_path,
    dose_percent_threshold, distance_threshold, local_gamma=True)

valid_gamma = gamma[~np.isnan(gamma)]
print(np.mean(valid_gamma < 1))

num_points = len(valid_gamma)
weights = np.ones(num_points)/num_points

plt.hist(valid_gamma, 20, weights=weights)
plt.title('Gamma histogram {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
plt.xlabel('Local Gamma')
plt.ylabel('Normalised bin count')

max_x_axis = np.ceil(np.max(valid_gamma))

plt.xlim([0, max_x_axis])
plt.show()

In [ ]:
num_points = len(valid_gamma)
weights = np.ones(num_points)/num_points

pass_rate = np.round(np.mean(valid_gamma < 1) * 100, decimals=1)

plt.figure(figsize=(8,5))

plt.hist(valid_gamma, 40, weights=weights, label="Gamma pass = {}%".format(pass_rate))
plt.title('Local Gamma histogram {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
plt.xlabel('Local Gamma')
plt.ylabel('Normalised bin count')

max_x_axis = np.ceil(np.max(valid_gamma))

plt.xlim([0, 2])
plt.legend()
plt.show()

In [ ]:
diff = dose_evaluation_40 - dose_reference
max_diff = np.max(np.abs(diff))
max_dose = np.max([np.max(dose_evaluation_40), np.max(dose_reference)])


In [ ]:
np.where(np.abs(diff) == max_diff)

In [ ]:

# i = 54

# nanned_dose_evaluation == 0


y, x = pcolormesh_grid(coords_reference[0], coords_reference[1])


def create_plots(i):
    print(i)
    plt.figure(figsize=(10,5))
    plt.pcolormesh(x, y, dose_reference[:,:,i], vmin=0, vmax=max_dose)
    plt.axis('equal')
    # plt.xlim([-400,-200])
    # plt.ylim([-20,200])
    cbar = plt.colorbar()
    cbar.set_label('Single fraction dose (Gy)')
    plt.gca().invert_yaxis()
    plt.title('Original planned beam')
    plt.xlabel('x (mm)')
    plt.ylabel('z (mm)')
    plt.show()

    plt.figure(figsize=(10,5))
    plt.pcolormesh(x, y, dose_evaluation_40[:,:,i], vmin=0, vmax=max_dose)
    plt.axis('equal')
    # plt.xlim([-400,-200])
    # plt.ylim([-20,200])
    cbar = plt.colorbar()
    cbar.set_label('Single fraction dose (Gy)')
    plt.gca().invert_yaxis()
    plt.title('Logfile produced beam')
    plt.xlabel('x (mm)')
    plt.ylabel('z (mm)')
    plt.show()

    plt.figure(figsize=(10,5))
    plt.pcolormesh(x, y, diff[:,:,i], vmin=-max_diff, vmax=max_diff, cmap='bwr')
    plt.axis('equal')
    # plt.xlim([-400,-200])
    # plt.ylim([-20,200])
    cbar = plt.colorbar()
    cbar.set_label('Single fraction dose difference (Gy)')
    plt.gca().invert_yaxis()
    plt.title('Dose difference (Logfile - Original)')
    plt.xlabel('x (mm)')
    plt.ylabel('z (mm)')
    plt.show()
    
    plt.figure(figsize=(10,5))
    plt.pcolormesh(x, y, gamma[:,:,i], vmin=0, vmax=2, cmap='coolwarm')
    plt.axis('equal')
    # plt.xlim([-400,-200])
    # plt.ylim([-20,200])
    cbar = plt.colorbar()
    cbar.set_label('Gamma')
    plt.gca().invert_yaxis()
    plt.title('Local Gamma {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
    plt.xlabel('x (mm)')
    plt.ylabel('z (mm)')
    plt.show()
    

for i in range(49,135):
    create_plots(i)